In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [2]:
class Library():
    catalogue=pd.DataFrame(columns=['name','item_format','creators','subject','availability','name_creators_format'])
    catalogue['availability'].dtypes==int
    def __init__(self,name):
        self.name=name
    
    def __str__(self):
        return "Managing catalogue and issue cards for {self.name}".format(self=self)

In [3]:
class User(Library):
    def __init__(self,name):
        super().__init__(name)
    
    def enter_details(self):
        name=input("Enter name: ")
        item_format=input("Enter item format: ")
        creators=input("Enter list of creator names: ")
        subject=input("Enter names of subjects: ")
        name_creators_format=name+'-'+creators+'-'+item_format
        return {'name':name,'item_format':item_format,'creators':creators,'subject':subject,'name_creators_format':name_creators_format}
    
    def search_item(self):
        response=''
        temp_dic=self.enter_details()
        name_creators_format=temp_dic['name_creators_format']
     
        count=0
        if (Library.catalogue['name_creators_format'].str.contains(name_creators_format).any()==True):
            count= (Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability'].iloc[0])
            if count>0:
                response="available"
            else:
                response='out of stock'
        elif (Library.catalogue['name_creators_format'].str.contains(name_creators_format).any()!=True):
            response="not available"
        return response,name_creators_format,count,temp_dic

In [4]:
class Customer(User):
    def __init__(self,name,username):
        super().__init__(name)
        self.username=username
        self.library_card=pd.DataFrame(columns=['Cust name','name','item_format','creators','subject','name_creators_format','Issue Date','Return Date','Status'])
        
    
    def borrow(self):
        response,name_creators_format,count,temp_dic=super().search_item()
        if response=='available':
            print("Issued")
            Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability']-=1
            temp_dic['Issue Date']=date.today().strftime("%d/%m/%Y")
            temp_dic['Return Date']=(date.today()+ datetime.timedelta(days=10)).strftime("%d/%m/%Y")
            temp_dic['Cust name']=self.username
            temp_dic['Status']='Borrowed'
            self.library_card=self.library_card.append(temp_dic,ignore_index=True)
            print(self.library_card)
        elif response=='out of stock':
            print("All on loan")
        elif response=='not available':
            print('not available in the library yet')   
        
    
    def return_item(self):
        response,name_creators_format,count,temp_dic=super().search_item()
        
        if (Library.catalogue['name_creators_format'].str.contains(name_creators_format).any()==True):
            Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability']+=1
            self.library_card.loc[self.library_card['name_creators_format']==name_creators_format,'Status']='Returned'
            self.library_card.loc[self.library_card['name_creators_format']==name_creators_format,'Return Date']=(date.today()).strftime("%d/%m/%Y")
            print(self.library_card)
        else:
            print("not from this library")
        
    
    def browse(self):
        browseby=input("Would you like to browse by creators, name or subject: ")
        browseby_spec=input("Enter name: ")
        column_name=[x for x in Library.catalogue.columns if x==browseby][0] 
        print(Library.catalogue.loc[Library.catalogue[column_name].str.contains(browseby_spec),:])
    
    def __str__(self):
        return "actions for customer {self.username}".format(self=self)




In [5]:
class Librarian(User):
    def __init__(self,name,librarian_name):
        super().__init__(name)
        self.librarian_name=librarian_name
   
    def add(self):
        response,name_creators_format,count,temp_dic=super().search_item()
        no_items=input("How many to add: ")
        
        if (response=='available') or (response=='out of stock'):
            Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability']+=int(no_items)
        elif (response=='not available'):
            Library.catalogue=Library.catalogue.append(temp_dic,ignore_index=True)
            Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability']=int(no_items)
        print(Library.catalogue)
    
    def remove(self):
        response,name_creators_format,count,temp_dic=super().search_item()
        no_items=input("How many to remove: ")
        if (response=='available'):
            Library.catalogue.loc[Library.catalogue['name_creators_format']==name_creators_format,'availability']-=int(no_items)
        elif (response=='not available'):
            print("Not in the library")
        elif (response=='out of stock'):
            print("all on loan")
        print(Library.catalogue)
    
    def __str__(self):
        return "actions of librarian {self.librarian_name}".format(self=self)

In [ ]:
play=''
while play!='yes' or play!='no':
    play=input("Do you want to enter library: ")
    
    if play=='yes':
        lib_name=input("Enter name of library: ")
        cust_name=input("Enter name of customer: ")
        librarian_name=input("Enter name of librarian: ")
        lpl=Library(lib_name)
        user=User(lib_name)
        cust=Customer(lib_name,cust_name)
        librarian=Librarian(lib_name,librarian_name)
        
        print(str(lpl))
        print(str(cust))
        print(str(librarian))
        
        while play=='yes':
            user=input("Are you librarian or customer: ")
            
            if user=='librarian':
                user_action=input("Do you want to add or remove: ")
                
                if user_action=='add':
                    librarian.add()
                elif user_action=='remove':
                    librarian.remove()
                else:
                    print("librarian can either add or remove")
                    continue
            
            elif user=='customer':
                user_action=input("Do you want to borrow, return or browse: ")
                
                if user_action=='borrow':
                    cust.borrow()
                elif user_action=='return':
                    cust.return_item()
                elif user_action=='browse':
                    cust.browse()
                else:
                    print("customer can either borrow, return or browse")
                    continue
            
            else:
                print("you can be either librarian or customer")
                continue
            play=input("Do you want to continue: ")
    elif play=='no':
        break
    else:
        print("enter yes or no")
        continue